# Part 17

# Using Functions

Universal code for the entire notebook

In [583]:
# Uncomment the line below to make sure you have all the packages needed
# %pip install -r requirements.txt

In [584]:
# Standard library imports

# Third-party library imports
from openai import OpenAI  # Used for interacting with OpenAI's API
from openai import AssistantEventHandler  # Used for handling events related to OpenAI assistants
from typing_extensions import override  # Used for overriding methods in subclasses
from IPython.display import display, Markdown, clear_output  # Used for displaying content in Jupyter Notebooks


In [585]:
# Create an instance of the OpenAI class to interact with the API.
# This assumes you have set the OPENAI_API_KEY environment variable.
client = OpenAI() 

In [586]:
# Define the functions the assistant will call
def get_current_temperature():
    # Simulate getting the current temperature
    return 57

def get_rain_probability():
    # Simulate getting the rain probability
    return 0.06

In [587]:
class EventHandler(AssistantEventHandler):
    """Custom event handler for processing assistant events."""

    def __init__(self):
        super().__init__()
        self.results = []  # Initialize an empty list to store the results

    @override
    def on_event(self, event):
        # Retrieve events that are denoted with 'requires_action'
        # since these will have our tool_calls
        if event.event == 'thread.run.requires_action':
            run_id = event.data.id  # Retrieve the run ID from the event data
            self.handle_requires_action(event.data, run_id)
    
    def handle_requires_action(self, data, run_id):
        tool_outputs = []
            
        for tool in data.required_action.submit_tool_outputs.tool_calls:
            if tool.function.name == "get_current_temperature":
                tool_outputs.append({"tool_call_id": tool.id, "output": str(get_current_temperature)})
            elif tool.function.name == "get_rain_probability":
                tool_outputs.append({"tool_call_id": tool.id, "output": str(get_rain_probability)})
        # Submit all tool_outputs at the same time
        self.submit_tool_outputs(tool_outputs, run_id)
        
    def submit_tool_outputs(self, tool_outputs, run_id):
        # Use the submit_tool_outputs_stream helper
        with client.beta.threads.runs.submit_tool_outputs_stream(
            thread_id=self.current_run.thread_id,
            run_id=self.current_run.id,
            tool_outputs=tool_outputs,
            event_handler=EventHandler(),
        ) as stream:
            for text in stream.text_deltas:
                print(text, end="", flush=True)
        
    @override
    def on_text_delta(self, delta, snapshot):
        """Handle the event when there is a text delta (partial text)."""
        # Append the delta value (partial text) to the results list
        self.results.append(delta.value)
        # Call the method to update the Jupyter Notebook cell
        self.update_output()

    def update_output(self):
        """Update the Jupyter Notebook cell with the current markdown content."""
        # Clear the current output in the Jupyter Notebook cell
        clear_output(wait=True)
        # Join all the text fragments stored in results to form the complete markdown content
        markdown_content = "".join(self.results)
        # Display the markdown content in the Jupyter Notebook cell
        display(Markdown(markdown_content))

In [588]:

assistant = client.beta.assistants.create(
    instructions="You are a weather bot. Use the provided functions to answer questions.",
    model="gpt-4o",
    tools=[
    {
        "type": "function",
        "function": {
            "name": "get_current_temperature",
            "description": "Get the current temperature for a specific location",
            "parameters": {
            "type": "object",
            "properties": {
                "location": {
                "type": "string",
                "description": "The city and state, e.g., San Francisco, CA"
                },
                "unit": {
                "type": "string",
                "enum": ["Celsius", "Fahrenheit"],
                "description": "The temperature unit to use. Infer this from the user's location."
                }
            },
            "required": ["location", "unit"]
            }
        }
        },
        {
        "type": "function",
        "function": {
            "name": "get_rain_probability",
            "description": "Get the probability of rain for a specific location",
            "parameters": {
            "type": "object",
            "properties": {
                "location": {
                "type": "string",
                "description": "The city and state, e.g., San Francisco, CA"
                }
            },
            "required": ["location"]
            }
        }
        }
    ]
)


In [589]:
thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="What's the weather in San Francisco today and the likelihood it will rain? Give me the results in a little table",
)

In [590]:
with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant.id,
    event_handler=EventHandler()
    ) as stream:
    stream.until_done()

| Weather Information  | Value                |
|----------------------|----------------------|
| Current Temperature  | 64°F                 |
| Probability of Rain  | 10%                  |

 |

In [591]:
import requests

def get_lat_lon(city_name, state_name=None):
    if state_name:
        location = f"{city_name},{state_name}"
    else:
        location = city_name
    
    geocode_url = f"https://geocode.xyz/{location}?json=1"
    response = requests.get(geocode_url)
    
    if response.status_code == 200:
        data = response.json()
        if 'latt' in data and 'longt' in data:
            return float(data['latt']), float(data['longt'])
        else:
            print("Error: Could not find latitude and longitude for the location.")
            return None, None
    else:
        print("Error: Geocoding request failed.")
        return None, None

def get_weather_forecast(lat, lon):
    points_url = f"https://api.weather.gov/points/{lat},{lon}"
    response = requests.get(points_url, headers={'User-Agent': 'MyWeatherApp (contact@example.com)'})
    
    if response.status_code == 200:
        data = response.json()
        forecast_url = data['properties']['forecast']
        
        forecast_response = requests.get(forecast_url, headers={'User-Agent': 'MyWeatherApp (contact@example.com)'})
        if forecast_response.status_code == 200:
            forecast_data = forecast_response.json()
            return forecast_data['properties']['periods'][0]['temperature']
        else:
            print("Error: Forecast request failed.")
            return None
    else:
        print("Error: Points request failed.")
        return None

def get_temperature(city_name, state_name=None):
    lat, lon = get_lat_lon(city_name, state_name)
    if lat is not None and lon is not None:
        temperature = get_weather_forecast(lat, lon)
        return temperature
    else:
        return None

# Example usage:
city_name = "New York"
state_name = "New York"
temperature = get_temperature(city_name, state_name)

if temperature is not None:
    print(f"The temperature in {city_name}, {state_name} is {temperature}°F")
else:
    print(f"Could not get the temperature for {city_name}, {state_name}")


The temperature in New York, New York is 75°F


In [608]:
class EventHandler(AssistantEventHandler):
    """Custom event handler for processing assistant events."""

    def __init__(self):
        super().__init__()
        self.results = []  # Initialize an empty list to store the results

    @override
    def on_event(self, event):
        # Retrieve events that are denoted with 'requires_action'
        # since these will have our tool_calls
        if event.event == 'thread.run.requires_action':
            run_id = event.data.id  # Retrieve the run ID from the event data
            self.handle_requires_action(event.data, run_id)
    
    def handle_requires_action(self, data, run_id):
        tool_outputs = []
            
        for tool in data.required_action.submit_tool_outputs.tool_calls:
            if tool.function.name == "get_temperature":
                tool_outputs.append({"tool_call_id": tool.id, "output": str(get_temperature)})
        # Submit all tool_outputs at the same time
        self.submit_tool_outputs(tool_outputs, run_id)
        
    def submit_tool_outputs(self, tool_outputs, run_id):
        # Use the submit_tool_outputs_stream helper
        with client.beta.threads.runs.submit_tool_outputs_stream(
            thread_id=self.current_run.thread_id,
            run_id=self.current_run.id,
            tool_outputs=tool_outputs,
            event_handler=EventHandler(),
        ) as stream:
            for text in stream.text_deltas:
                print(text, end="", flush=True)
        
    @override
    def on_text_delta(self, delta, snapshot):
        """Handle the event when there is a text delta (partial text)."""
        # Append the delta value (partial text) to the results list
        self.results.append(delta.value)
        # Call the method to update the Jupyter Notebook cell
        self.update_output()

    def update_output(self):
        """Update the Jupyter Notebook cell with the current markdown content."""
        # Clear the current output in the Jupyter Notebook cell
        clear_output(wait=True)
        # Join all the text fragments stored in results to form the complete markdown content
        markdown_content = "".join(self.results)
        # Display the markdown content in the Jupyter Notebook cell
        display(Markdown(markdown_content))

In [609]:
current_temperature_assistant = client.beta.assistants.create(
    instructions="You are a weather bot. To get the temperature, use the get_temperature function.",
    model="gpt-4o",
    tools=[
    {
        "type": "function",
        "function": {
            "name": "get_temperature",
            "description": "Get the current temperature for a specific location.",
            "parameters": {
            "type": "object",
            "properties": {
                "city_name": {
                "type": "string",
                "description": "The city (e.g. San Francisco)"
                },
                "state_name": {
                "type": "string",
                "description": "The state related to the city (e.g. CA)"
                },
            },
            "required": ["city_name"]
            }
        }
        },
        
    ]
)

In [610]:
current_temperature_thread = client.beta.threads.create()
current_temperature_message = client.beta.threads.messages.create(
    thread_id=current_temperature_thread.id,
    role="user",
    content="What's the weather in New York, NY today? Give me the result in a little table.",
)

In [611]:
with client.beta.threads.runs.stream(
    thread_id=current_temperature_thread.id,
    assistant_id=current_temperature_assistant.id,
    event_handler=EventHandler()
    ) as stream:
    stream.until_done()

I am currently unable to retrieve the weather information. Please try again later or check a reliable weather website or app for the current conditions in New York, NY.

.